# Step 0: setting up the folders
We'll first the folders and videofiles, extract the audio from it, and initalize some packages.

In [48]:
# Step 0: Setting up the folders and files
import mediapipe as mp #mediapipe
import cv2 #opencv
import math #basic operations
import numpy as np #basic operations
import pandas as pd #data wrangling
import csv #csv saving
import os #some basic functions for inspecting folder structure etc.
from os import listdir
from os.path import isfile, join
import glob as glob
%pip install moviepy
import moviepy.editor as mop

#list all videos in input_videofolder
myinputpath = "./inputvideo/" #this is your folder with (all) your video(s)
myoutputpath = "./Output_Videos/" #this is the folder where the masked videos will be saved
#list all filenames that has .mp4 or .avi or .mov extensions
vfiles = [f for f in listdir(myoutputpath) if isfile(join(myoutputpath, f)) and f.endswith(('.mp4', '.avi', '.mov'))]
#vfiles = [f for f in "./Output_Videos"]    
#time series output folder
inputfol = "./inputvideo/"
output = "./Output_Videos/"
outputf = "./Animated_Videos/"
outtputf_ts = "./TS_processed/"


#check videos to be processed
print("The following folder is set as the output folder where all the pose time series are stored")
print(os.path.abspath(outtputf_ts))
print("\n The following folder is set as the output folder for saving the masked videos ")
print(os.path.abspath(outputf))
print("\n The following video(s) will be processed for masking: ")
print(vfiles)

Note: you may need to restart the kernel to use updated packages.
The following folder is set as the output folder where all the pose time series are stored
c:\Users\kadava\Documents\Github\FLESH_IteratedLearning\TS_processed

 The following folder is set as the output folder for saving the masked videos 
c:\Users\kadava\Documents\Github\FLESH_IteratedLearning\Animated_Videos

 The following video(s) will be processed for masking: 
['Donner_g_ch1_g10_compr_clean.mp4', 'Donner_g_ch1_g11_compr_clean.mp4', 'Donner_g_ch1_g12_compr_clean.mp4', 'Donner_g_ch1_g13_compr_clean.mp4', 'Donner_g_ch1_g14_compr_clean.mp4', 'Donner_g_ch1_g15_compr_clean.mp4', 'Donner_g_ch1_g16_compr_clean.mp4', 'Donner_g_ch1_g17_compr_clean.mp4', 'Donner_g_ch1_g18_compr_clean.mp4', 'Donner_g_ch1_g19_compr_clean.mp4', 'Donner_g_ch1_g1_compr_clean.mp4', 'Donner_g_ch1_g20_compr_clean.mp4', 'Donner_g_ch1_g2_compr_clean.mp4', 'Donner_g_ch1_g3_compr_clean.mp4', 'Donner_g_ch1_g4_compr_clean.mp4', 'Donner_g_ch1_g5_compr_clea

# Step 3: Animating a video
Now we are arriving at the fun part. We have our motion tracking performed and saved as a video (and time series), and our amplitude envelope function ready as well. Now we simply need to loop over the motion tracking video frames, and generate a plot that shows the movement speed and the envelope data for that moment, plus some time before and after. We set the time window for 4 seconds, where we show -2 and plus 2 seconds relative to the current moment in the video.

Note that this does take some time rendering. This is because you need to create a plot for every frame, and write a video with the plot now added to the masked video. 

In [47]:
import tempfile
import shutil
import tqdm
import cv2
import matplotlib.pyplot as plt
import numpy as np
# guassian filter
from scipy.ndimage import gaussian_filter

# what is the window size in seconds
window = 4

# a function that generates a plot containing two panel time series of the envelope (panel 1) and the pose time series (panel 2) with a window of 4 seconds
def plot_envelope_pose(body, midpoint, g1v1, g1v2, g2v1, g2v2, minbody1, maxbody1, minbody2, maxbody2):
    # make a temporary folder
    tempfolder = tempfile.mkdtemp()
    fig, ax = plt.subplots(2, 1, figsize=(14, 10))
    # selection 4 seconds from midpoint
    start = midpoint - window/2
    end = midpoint + window/2
    # if start is negative, set it to 0
    if start < 0:
        start = 0
    # subset the body time series and amplitude envelope based on the start and end
    body = body[(body['time'] >= start) & (body['time'] <= end)]
    #audio = audio[(audio['time'] >= start) & (audio['time'] <= end)]
    # do a cross correlation between the envelope and the pose time series
    # recenter time
    body['time'] = body['time'] - midpoint
    #audio['time'] = audio['time'] - midpoint
    # Plot the amplitude envelope
    ax[0].plot(body['time'], body[g1v1], label=g1v1, color='magenta', alpha=0.8, linewidth=4)
    ax[0].plot(body['time'], body[g1v2], label=g1v2, color='darkblue', alpha=0.8, linewidth=4)
    ax[0].legend() # show labels in legend
    ax[0].legend(prop={'size': 24})
    # Plot the pose time series with a thick line semitransparent, but different colors (black & grey)
    ax[1].plot(body['time'], body[g2v1], label=g2v1, color='magenta', alpha=0.8, linewidth=4)
    ax[1].plot(body['time'], body[g2v2], label=g2v2, color='darkblue', alpha=0.8, linewidth=4)
    ax[1].set_xlabel('Time (s)', fontsize=24)
    ax[1].legend() # show labels in legend
    ax[1].legend(prop={'size': 24})
    plt.tight_layout()
    # set the x axes to centered by 0 minus and plus half the window
    ax[0].set_xlim(-window/2, window/2)
    ax[1].set_xlim(-window/2, window/2)
    # add a vertical line in the plot at 0
    ax[0].axvline(x=0, color='r', linestyle='--', linewidth=4, alpha=0.8)
    ax[1].axvline(x=0, color='r', linestyle='--', linewidth=4, alpha=0.8)
    # set the y axes - taken from sarka - right now it does it for each frame, but thats less ideal. 
    #minbody1 = min(body[g1v1].min(), body[g1v2].min())
    #maxbody1 = max(body[g1v1].max(), body[g1v2].max())
    #maxbody2 = max(body[g2v1].max(), body[g2v2].max())
    #minbody2 = min(body[g2v1].min(), body[g2v2].min())  
    ax[0].set_ylim(minbody1, maxbody1)
    ax[1].set_ylim(minbody2, maxbody2)
    # increase font size
    for a in ax:
        a.tick_params(axis='both', which='major', labelsize=24)
        a.tick_params(axis='both', which='major', width=6)
    # save the plot
    tpf = tempfolder + 'tempfig.png'
    plt.savefig(tpf )
    plt.close()
    img = cv2.imread(tpf)
    shutil.rmtree(tempfolder)
    return img


# we loop over the video files and generate animated videos (with and without sound)
for vidf in vfiles:	
    videofilename = output + vidf
    outputfilename = outputf + vidf
    #audiofilename = inputfol + vidf.replace(".mp4", ".wav")
    #videofilenamemasked = videofilename.replace(".mp4", "_masked.mp4")
    print(videofilename)
    # Get the amplitude envelope
    #ampv, sr = amp_envelope(audiofilename)
    # get the raw audio
    #rawaudio, sr = librosa.load(audiofilename, sr=None)
    # save the audio and envelope in a pandas with time in seconds
    #audio = pd.DataFrame({'time': np.arange(0, len(rawaudio)/sr, 1/sr), 'audio': rawaudio, 'envelope': ampv})
    # Load the pose time series
    #body = pd.read_csv(outtputf_ts + vidf[:-4] + '_body.csv')
    body = pd.read_csv(outtputf_ts + vidf[:-4] + ".csv")
    # now calculate the right hand and left hand speed
    body['time'] = body['time']/1000
    g1v1 = 'LEFT_WRIST_speed'	
    g1v2 = 'RIGHT_WRIST_speed'
    g2v1 = 'Y_LEFT_WRIST_velocity'
    g2v2 = 'Y_RIGHT_WRIST_velocity'
    minbody1 = min(body[g1v1].min(), body[g1v2].min())
    maxbody1 = max(body[g1v1].max(), body[g1v2].max())
    maxbody2 = max(body[g2v1].max(), body[g2v2].max())
    minbody2 = min(body[g2v1].min(), body[g2v2].min()) 
    # interpolate nan using lambda, first set nan to NaN
    #body = body.replace('nan', np.nan)
    #  # approx nas using lambda
    #body = body.apply(lambda x: x.interpolate(method='linear') if x.name != 'time' else x)
    # smooth all the time series using lambda using guassian
    #body = body.apply(lambda x: gaussian_filter(x, 3) if x.name != 'time' else x)
    # calculate the speed of the left and right hand
    #body['left_wrist_speed'] = np.insert(np.sqrt(np.diff(body['X_LEFT_WRIST'])**2 + np.diff(body['Y_LEFT_WRIST'])**2),0,0)
    #body['right_wrist_speed'] = np.insert(np.sqrt(np.diff(body['X_RIGHT_WRIST'].diff())**2 + np.diff(body['Y_RIGHT_WRIST'])**2),0,0)
        # Normalize the speed to 0-1 by the min/max right hand speed 
    #body['left_wrist_speed'] = (body['left_wrist_speed'] - np.min(body['left_wrist_speed'])) / (np.max(body['left_wrist_speed']) - np.min(body['left_wrist_speed']))
    #body['right_wrist_speed'] = (body['right_wrist_speed'] - np.min(body['right_wrist_speed'])) / (np.max(body['right_wrist_speed']) - np.min(body['right_wrist_speed']))
    # load the video in opencv and prepare to loop over it
    capture = cv2.VideoCapture(videofilename)
    frameWidth = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
    frameHeight = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
    fps = capture.get(cv2.CAP_PROP_FPS)
    # out
    fourcc = cv2.VideoWriter_fourcc(*'MP4V')
    out = cv2.VideoWriter(outputfilename[:-4] + '_animated.mp4', fourcc, fps,  (int(frameWidth)*4, int(frameHeight)))
    # loop over the video
        # loop through the video and add the plot to the video on left upper corner in small inset
    frame_number = 0 
    # with progress bar
    for i in tqdm.tqdm(range(int(capture.get(cv2.CAP_PROP_FRAME_COUNT)))):
        ret, frame = capture.read()
        if ret == True:
            # change the variable names in the fuction below to graph different variables
            img = plot_envelope_pose(body, frame_number/fps, g1v1, g1v2, g2v1, g2v2, minbody1, maxbody1, minbody2, maxbody2)
            # resize the image
            img = cv2.resize(img, (int(frameWidth*3), int(frameHeight)))
            # lets put the plot on the right side of the frame using concat
            frame = np.concatenate([frame, img], axis=1)
            out.write(frame)
            frame_number += 1
            # also show the frame (optional)
            #cv2.imshow('Frame', frame)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
        else:
            break
    capture.release()
    out.release()    
    cv2.destroyAllWindows()
    break

./Output_Videos/Donner_g_ch1_g10_compr_clean.mp4


100%|██████████| 419/419 [01:26<00:00,  4.84it/s]


In [30]:
outputf

'./Testing_Animated_Videos/'

In [38]:
for vidf in vfiles:
    videofilename = outputf + vidf[:-4] + '_animated.mp4'
    #audiofilename = inputfol + vidf.replace(".mp4", ".wav")
    video = mop.VideoFileClip(videofilename)
    #audio = mop.AudioFileClip(audiofilename)
    #video = video.set_audio(audio)
    video.close()
    break

# show the video in the notebook
from IPython.display import Video
Video(outputf + vfiles[0][:-4] + '_animated.mp4', width=800, embed = True)

KeyError: 'video_fps'

In [31]:
from IPython.display import Video
Video(outputf + vfiles[0][:-4] + '_animated.mp4', width=800, embed = True)